# ყველა ფაილის მისამართის ამოღება

In [1]:
!pip install librosa
!pip install pydub

In [27]:
import glob
import os
from pydub import AudioSegment

# audio ფაილების გადაყვანა wav-ში

In [ ]:
def convert_to_wav(files):
    for src in files:
        dst = src[:-3] + "wav"
        s = AudioSegment.from_file(src)
        s.export(dst, format="wav")

In [65]:
folders = os.listdir('data')
for sub_dir in folders:
#     print(sub_dir)
    mp3_files = glob.glob(os.getcwd() + '/data/' + sub_dir + '/*mp3')
    convert_to_wav(mp3_files)
    m4a_files = glob.glob(os.getcwd() + '/data/' + sub_dir + '/*m4a')
#     print(len(m4a_files))
#     რატომღაც ამას არ შვება :(
#     convert_to_wav(m4a_files) 

In [9]:
import pandas as pd
import librosa
import librosa.display

In [66]:
import pandas as pd
import os
import librosa
# უბრალოდ შევამოწმე რას აკეთებდა

features = []
data = extract_features("test.wav")
data2 = extract_features("devi-khos-1a.wav")
label = "1"
features.append([data, label])
features.append([data2, label])
datafr = pd.DataFrame(features, columns=['feature','class_label' ])
datafr

NameError: name 'extract_features' is not defined

##  <font color =green>extract_features ვარგა. ქვემოთ რაც წერია ყველაფერს სწორად შვება</font>

In [11]:
import numpy as np

def get_features(file_name):
   
    try:
        librosa.load
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [24]:
def get_all_features():
    folders = os.listdir('data')
    folders.sort()
    features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.path.join('data',sub_dir,"*.wav")):
            print("Extracting file ", file_name)
            try:
                mfccs = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            features.append([mfccs,label])
    data = pd.DataFrame(features, columns=['feature', 'class_label'])
    return data

In [14]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccsscaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, 61), mfccsscaled - (40,)
    return mfccsscaled

In [15]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
#     rate - (1, 61)
    return rate

In [16]:
def get_spectral_centroid(audio):
    #SPECTRAL_CENTROID
    sp = librosa.feature.spectral_centroid(audio)
#     sp - (1, 61)
    return sp

In [17]:
def get_spectral_roll_ff(audio, sr):
    # Spectral-roll off
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
#     a - (1, 61)
    return a

In [18]:
def get_spectral_flux(audio, sr):
    # Spectral flux
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
#     onset_env - (61,)
    return onset_env

In [19]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, 61)
    return [chroma_stft, chroma_cens, chroma_cq]

In [20]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, 61)
    return pitches

In [ ]:
# get_all_features()

# მინაწერი :დ